This work is inspired by paper https://arxiv.org/pdf/1703.10593.pdf, however, we don't attempt to duplicate their exact code and network architecture here.

In [2]:
# from __future__ import print_function
# %matplotlib inline
# import argparse
# import os
import random
import torch
import torch.nn as nn
# import torch.nn.parallel
# import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
# import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
# import matplotlib.animation as animation
# from IPython.display import HTML

# Set random seed for reproducibility
manualSeed = 999
#manualSeed = random.randint(1, 10000) # use if you want new results
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

Random Seed:  999


In [6]:
class Arguments():
    # main parameters

    # Root directory for dataset
    dataroot = "data/train"

    # Number of workers for dataloader
    workers = 4

    # Batch size during training
    batch_size = 32

    # Spatial size of training images. All images will be resized to this
    #   size using a transformer.
    image_size = 256

    # Number of channels in the group A
    input_nc = 3
    
    # Number of channels in the group B
    output_nc = 3

    # number of filters in the last conv layer of generator
    ngf = 8

    # number of filters in the first conv layer of discriminator
    ndf = 8

    # Number of training epochs
    num_epochs = 5

    # Learning rate for optimizers
    lr = 0.0002
    
    # identity_loss coeff.
    lambda_identity = 0.1

    # Beta1 hyperparam for Adam optimizers
    beta1 = 0.5

    # Number of GPUs available. Use 0 for CPU mode.
    ngpu = 1

    device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")
    print(device)
    
opt = Arguments()

cpu


### We need two generators and two discriminators, here we define them

In [7]:
class Generator(nn.Module):
    def __init__(self):
        """
        Parameters:
            input_nc (int) -- the number of channels in input images
            output_nc (int) -- the number of channels in output images
            ngf (int) -- the number of filters in the last conv layer
        Returns a generator
        """
        super(Generator, self).__init__(input_nc, output_nc, ngf)
        self.main = nn.Sequential(
            # Down sampling
            nn.Conv2d(input_nc, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.LeakyReLU(0.2, inplace=True),
            # state size.  ngf x (image_size/2) x (image_size/2)
            
            nn.Conv2d(ngf, ngf*2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. ngf*2 x (image_size/4) x (image_size/4)
            
            nn.Conv2d(ngf*2, ngf*4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. ngf*4 x (image_size/8) x (image_size/8)
            
            # ---
            # Up sampling
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. ngf*2 x (image_size/4) x (image_size/4)
            
            nn.ConvTranspose2d(ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. (ngf) x image_size/2 x image_size/2
            
            nn.ConvTranspose2d(ngf, output_nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. (output_nc) x image_size x image_size
        )

    def forward(self, input):
        return self.main(input)

In [8]:
# Discriminator Code    
class Discriminator(nn.Module):
    def __init__(self):
        """
        Parameters:
            input_nc (int)  -- the number of channels in input images
            ndf (int)       -- the number of filters in the last conv layer
        """
        super(Discriminator, self).__init__(input_nc, ndf)
        self.main = nn.Sequential(
            # input is (input_nc) x image_size x image_size
            nn.Conv2d(input_nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x image_size/2 x image_size/2
            
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x image_size/4 x image_size/4
            
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x image_size/8 x image_size/8
            
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*8) x image_size/16 x image_size/16
            
            nn.Conv2d(ndf * 8, ndf * 16, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 16),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*16) x image_size/32 x image_size/32
            
            nn.Conv2d(ndf * 16, ndf * 32, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 32),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*32) x image_size/64 x image_size/64
            
            nn.Conv2d(ndf * 32, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)

In [ ]:
class CycleGANModel():
    def __init__(self, opt):
        # specify the training losses you want to print out.
        self.loss_names = ['D_A', 'G_A', 'cycle_A', 'idt_A', 'D_B', 'G_B', 'cycle_B', 'idt_B']
        
        # specify the images you want to save/display.
        visual_names_A = ['real_A', 'fake_B', 'rec_A', 'idt_B']
        visual_names_B = ['real_B', 'fake_A', 'rec_B', 'idt_A']
        
        self.visual_names = visual_names_A + visual_names_B  # combine visualizations for A and B
        
        self.model_names = ['G_A', 'G_B', 'D_A', 'D_B']
        
        # define networks (both Generators and discriminators)
        self.netG_A = Generator(opt.input_nc, opt.output_nc, opt.ngf)
        self.netG_B = Generator(opt.output_nc, opt.input_nc, opt.ngf)
        
        self.netD_A = Discriminator(opt.output_nc, opt.ndf)
        self.netD_B = Discriminator(opt.input_nc, opt.ndf)
        
#         self.fake_A_pool = ImagePool(opt.pool_size)  # create image buffer to store previously generated images
#         self.fake_B_pool = ImagePool(opt.pool_size)  # create image buffer to store previously generated images
            
        # define loss functions
#         self.criterionGAN = ? # define GAN loss.
        self.criterionCycle = torch.nn.L1Loss()
        self.criterionIdt = torch.nn.L1Loss()
        
        # initialize optimizers; schedulers will be automatically created by function <BaseModel.setup>.
        self.optimizer_G = torch.optim.Adam(itertools.chain(self.netG_A.parameters(), self.netG_B.parameters()), lr=opt.lr, betas=(opt.beta1, 0.999))
        self.optimizer_D = torch.optim.Adam(itertools.chain(self.netD_A.parameters(), self.netD_B.parameters()), lr=opt.lr, betas=(opt.beta1, 0.999))
        self.optimizers.append(self.optimizer_G)
        self.optimizers.append(self.optimizer_D)
        
    def set_input(self, input):
        """Unpack input data from the dataloader"""
        self.real_A = input['A']
        self.real_B = input['B']
        
    def forward(self):
        """Run forward pass"""
        self.fake_B = self.netG_A(self.real_A)  # G_A(A)
        self.rec_A = self.netG_B(self.fake_B)   # G_B(G_A(A))
        self.fake_A = self.netG_B(self.real_B)  # G_B(B)
        self.rec_B = self.netG_A(self.fake_A)   # G_A(G_B(B))
        
    